In [1]:
# Importamos las librerías necesarias para el proyecto
import pandas as pd

In [2]:
# Cargamos la base de datos principal
df = pd.read_csv('../data/indicadores_generales_distritos.csv', sep=';', encoding='utf-8-sig')

In [3]:
# Configuramos la visualización del dataframe para mostrar todas las columnas
pd.set_option('display.max_columns', None)
df

,Orden,Periodo panel,ciudad,cod_distrito,distrito,cod_barrio,barrio,año,fecha_indicador,fuente_indicador,categoría_1,categoría_2,indicador_nivel1,indicador_nivel2,indicador_nivel3,unidad_indicador,indicador_completo,valor_indicador
0,1,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Características Generales del Distrito-Barrio,NaN,Superficie,NaN,NaN,Ha,Superficie (Ha.),60.446
1,2,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Características Generales del Distrito-Barrio,NaN,Densidad de población,NaN,NaN,Habitantes/Ha.,Población densidad (hab./Ha.),54
2,3,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,NaN,NaN,Habitantes,Número Habitantes,3.266.126
3,4,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,Hombres,NaN,Habitantes,Población Hombres,1.521.178
4,5,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,Mujeres,NaN,Habitantes,Población Mujeres,1.744.948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69540,69541,2023,Ciudad de Madrid,21.0,Barajas,211.0,Alameda de Osuna,2022,01/01/2022,Estadística Ayuntamiento de Madrid,Vivienda,NaN,Inmuebles,Uso residencial,Superficie media,m2,Superficie media construida (m2) inmuebles de ...,"126,39"
69541,69542,2023,Ciudad de Madrid,21.0,Barajas,212.0,Aeropuerto,2022,01/01/2022,Estadística Ayuntamiento de Madrid,Vivienda,NaN,Inmuebles,Uso residencial,Superficie media,m2,Superficie media construida (m2) inmuebles de ...,"63,81"
69542,69543,2023,Ciudad de Madrid,21.0,Barajas,213.0,Casco Histórico de Barajas,2022,01/01/2022,Estadística Ayuntamiento de Madrid,Vivienda,NaN,Inmuebles,Uso residencial,Superficie media,m2,Superficie media construida (m2) inmuebles de ...,"96,15"
69543,69544,2023,Ciudad de Madrid,21.0,Barajas,214.0,Timón,2022,01/01/2022,Estadística Ayuntamiento de Madrid,Vivienda,NaN,Inmuebles,Uso residencial,Superficie media,m2,Superficie media construida (m2) inmuebles de ...,"128,07"


#### Limpieza de datos inicial

In [4]:
# Chequeamos el tipo de datos de las columnas y los valores nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69545 entries, 0 to 69544
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Orden               69545 non-null  int64  
 1   Periodo panel       69545 non-null  int64  
 2   ciudad              69545 non-null  object 
 3   cod_distrito        68464 non-null  float64
 4   distrito            68464 non-null  object 
 5   cod_barrio          50804 non-null  float64
 6   barrio              50812 non-null  object 
 7   año                 69545 non-null  int64  
 8   fecha_indicador     69545 non-null  object 
 9   fuente_indicador    69545 non-null  object 
 10  categoría_1         69545 non-null  object 
 11  categoría_2         61903 non-null  object 
 12  indicador_nivel1    69545 non-null  object 
 13  indicador_nivel2    60506 non-null  object 
 14  indicador_nivel3    26899 non-null  object 
 15  unidad_indicador    69526 non-null  object 
 16  indi

In [5]:
# Comprobamos que no haya duplicados
df.duplicated().sum()

np.int64(0)

In [6]:
df.nunique()

Orden                 69545
Periodo panel             4
ciudad                    1
cod_distrito             21
distrito                 32
cod_barrio              131
barrio                  206
año                      12
fecha_indicador          19
fuente_indicador         14
categoría_1              19
categoría_2              51
indicador_nivel1         44
indicador_nivel2        204
indicador_nivel3         78
unidad_indicador         39
indicador_completo      410
valor_indicador       19492
dtype: int64

In [7]:
# Eliminar espacios al comienzo del nombre de algunos distritos
df['distrito'] = df['distrito'].str.strip()

In [8]:
# Correccion de tipo de datos para poder utilizar las columnas más adelante
df['fecha_indicador'] = pd.to_datetime(df['fecha_indicador'], format='%d/%m/%Y', errors='coerce')

In [9]:
# Adaptamos el formato de puntuación en los valores para una correcta interpretación en la conversión a números
df['valor_indicador'] = df['valor_indicador'].str.replace('.', '', regex=False)
df['valor_indicador'] = df['valor_indicador'].str.replace(',', '.', regex=False)

In [10]:
# Convertir a numérico el valor de los indicadores
df['valor_indicador'] = pd.to_numeric(df['valor_indicador'], errors='coerce')

#### Resumen de indicadores en la tabla

In [11]:
# Visualizamos las categorías generales y más específicas
df.groupby('categoría_1')['categoría_2'].value_counts()

categoría_1                             categoría_2                                              
Calidad de vida                         Satisfacción con espacios públicos                             572
                                        Calidad de vida y agenda pública                               264
                                        Percepción de la seguridad                                     264
                                        Principales poblemas de la ciudadanía                          264
                                        Problemas que afectan a la seguridad                           166
                                        Administración de la ciudad                                    154
                                        Amigabilidad de la ciudad                                       88
                                        Ciudadanía                                                      44
Educación                               Poblac

In [12]:
#Lista de todos los indicadores que están en esta base de datos
df['indicador_completo'].unique()

array(['Superficie (Ha.) ', 'Población densidad (hab./Ha.) ',
       'Número Habitantes', 'Población Hombres ', 'Población Mujeres',
       'Edad media de la población', 'Población de 0 a 14 años   ',
       'Población de 15 a 29 años', 'Población de 30 a 44  años',
       'Población de 45 a 64 años', 'Población de 65 a 79 años      ',
       'Población de 80 años y más           ',
       'Población de 65 años y más          ',
       'Población en etapa educativa (Población de 0 a 16 años -16 no incluidos)                            ',
       'Proporción de envejecimiento (Población mayor de 65 años/Población total)',
       'Proporción de sobre-envejecimiento (Población mayor de 80 años/ Población mayor de 65 años)',
       'Índice de dependencia (Población de 0-15 + población 65 años y más / Pob. 16-64) ',
       'Personas con nacionalidad española             ',
       'Personas con nacionalidad española Hombres                      ',
       'Personas con nacionalidad española Mu

#### Extraer datos por distrito

In [13]:
#Crear un dataframe con los códigos y nombres de los distritos
df_distritos = df[['cod_distrito', 'distrito']].drop_duplicates().dropna()
df_distritos

,cod_distrito,distrito
274,1.0,Centro
1047,2.0,Arganzuela
1884,3.0,Retiro
2636,4.0,Salamanca
3378,5.0,Chamartín
4125,6.0,Tetuán
4887,7.0,Chamberí
5638,8.0,Fuencarral-El Pardo
6579,9.0,Moncloa-Aravaca
7411,10.0,Latina
